# Gait Classification 

In this notebook we are going to achieve the following:

1. **Create the Training Dataset**: We will preprocess and combine data from different sources to create a comprehensive training dataset.
2. **Feature Selection and Dimensionality Reduction**: We will identify the most relevant features for classification using dimensionality reduction techniques.
3. **Model Evaluation**: We will test and compare the performance of multiple machine learning and deep learning algorithms for gait classification.

In [ ]:
# Libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pointbiserialr
from data_preprocessing import classification_merge_all_types
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut, cross_val_score

# Constants
healthy_dir = 'Data/Healthy'
stroke_dir = 'Data/Stroke'

In [ ]:
# Create the data file combining the results of feature extraction for both healthy and stroke patients
classification_merge_all_types(healthy_dir, stroke_dir)
data = pd.read_csv('final_dataset.csv')


# Drop the column subject_id as we don't need it
data = data.drop(columns=['subject_id'])
data.head()

# Feature Selection and Dimensionality Reduction

## 1. Point-biserial Correlation
The point biserial correlation coefficient is a measure of the correlation between a binary variable (such as a yes/no or pass/fail variable) and a continuous variable. It is similar to the Pearson correlation coefficient, but is used specifically for this type of data. The point biserial correlation coefficient ranges from -1 to 1, with positive values indicating a positive correlation and negative values indicating a negative correlation. Values close to 0 indicate little or no correlation. The p-value represents the probability that the correlation between the two variables is due to chance. Typically, a p-value of less than 0.05 is considered to be statistically significant.

In [ ]:
label = data['label']
features = data.drop(columns=['label'])

correlations, p_values = [], []

# Calculate the Point Biserial Correlation Coefficient for each feature
for feature in features:
    correlation, p_value = pointbiserialr(label, features[feature])
    correlations.append(correlation)
    p_values.append(p_value)

# Sort the features by their correlation with the label and create dataframes for better visualization
sorted_indices = np.argsort(correlations)[::-1]
sorted_features = features.columns[sorted_indices]
sorted_correlations = np.array(correlations)[sorted_indices]
sorted_p_values = np.array(p_values)[sorted_indices]

corr_df = pd.DataFrame({'Feature': sorted_features, 'Correlation': sorted_correlations, 'p-value': sorted_p_values})
corr_df.dropna()
corr_df = corr_df[corr_df['p-value'] < 0.05]
corr_df

Based on the results here we know that the feature **right-z-axis-(deg/s)-min** has a strong positive correlation of 0.77 and a p-value of 0.001 ensuring that this correlation is not the result of luck. 

---

## 2. Feature Correlation Check though Correlation Matrix


In [ ]:
# Create a heatmap of the correlation matrix, excluding the label column that we know have Nan values
columns = ['left-z-axis-(deg/s)-kurt', 'left-z-axis-(deg/s)-zcr', 'right-z-axis-(deg/s)-zcr', 'right-z-axis-(deg/s)-pkcnt', 'left-z-axis-(deg/s)-pkcnt']
data = data.drop(columns=columns)

plt.figure(figsize=(16, 16))
axis_corr = sns.heatmap(data.corr(), vmin=-1, vmax=1, center=0, cmap='coolwarm', square=True, linewidths=0.1, annot_kws={'fontsize':10})
plt.show()

---

## 3. Principle Component Analysis

The result of PCA gives us a clear separability between the classes of Healthy and Stroke subjects, proving that the two classes can be classified effectively. 
| **Pros of PCA**                  | **Why It’s Useful**                          |
| -------------------------------- | -------------------------------------------- |
| Reveals structure in the data    | Shows healthy vs stroke are separable        |
| Reduces dimensions               | Useful for visualization and fast models     |
| Can improve performance          | Helps in small datasets if too many features |
| Less overfitting risk            | Fewer dimensions = less noise/flexibility    |

In [ ]:

X = data.drop(columns=['label'])
y = data['label']

# Step 2: Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA (2 components for visualization)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Step 4: Plot the PCA projection
pca_df = pd.DataFrame(X_pca, columns=['PC1', 'PC2'])
pca_df['label'] = y

plt.figure(figsize=(8, 6))
sns.scatterplot(data=pca_df, x='PC1', y='PC2', hue='label', palette='Set2', s=100)
plt.title('PCA Projection of Gait Features')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.grid(True)
plt.show()

One of the main issue with using PCA is that the information is not useful in this contexts. So although PCA easily helps us classify the users, we can not interpret the data and provide insights that doctors can use to help the patients.

| **Cons of PCA (in this use)**        | **Why It’s a Problem**                            |
| -----------------------------------  | ------------------------------------------------- |
| PC1/PC2 are **not interpretable**    | They are weighted sums of 50+ original features   |
| No physical or clinical meaning      | We can’t say PC1 means stride time variability    |
| Clinicians can’t act on it           | They want to know *which* aspect of gait is wrong |
| Feature importance is hidden         | We lose clarity on what the model is using       |

---

## 4. Random Forest and Recursive Feature Elimination  

Recursive Feature Elimination fits a model like Random Forest that starts with all the input variables, then iteratively removes those with the weakest relationship with the output until the desired number of features is reached. It actually fits a model instead of just running statistical tests unlike the Univariate Testing. This is done in order to drop highly correlated features. Given that features X1 and X2 are highly correlated then they will have the same kind of effect on the output variable, where the change in performance observed tends to be negligible. 

The idea here is to reduce the computational complexity and the noise of data while again retaining the meaningful information. 

In [ ]:
# Load dataset
df = pd.read_csv('final_dataset.csv')
X = df.drop(columns=['subject_id', 'label'])
y = df['label']

# Standardize features and Split dataset
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=42)

# Train RandomForest and get the feature importance
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
importance_df = pd.DataFrame({'feature': X.columns, 'importance': rf.feature_importances_}).sort_values(by='importance', ascending=False)

# Recursive Feature Elimination and get feature importance
rfe = RFE(estimator=rf, n_features_to_select=10)
rfe.fit(X_train, y_train)
rfe_df = pd.DataFrame({'feature': X.columns, 'selected_by_rfe': rfe.support_, 'ranking': rfe.ranking_}).sort_values(by='ranking')

# Merge results
merged = pd.merge(importance_df, rfe_df, on='feature', how='inner', validate='one_to_one')
merged.sort_values(by='ranking')

Based on the feature selection above we found that 11 out of the 58 features provide us the best subset of data to classify between the Healthy and Stroke subjects. The features that provide us the best chance are the following:

| Feature                     | Source                        |
|-----------------------------|-------------------------------|
| right-z-axis-(deg/s)-min    | Point-biserial Correlation    |
| left-z-axis-(deg/s)-iqr     | Recursive Feature Elimination |
| left-z-axis-(deg/s)-min     | Recursive Feature Elimination |
| left-z-axis-(deg/s)-rms     | Recursive Feature Elimination |
| left_gait_band_energy_mean  | Recursive Feature Elimination |
| left-z-axis-(deg/s)-std     | Recursive Feature Elimination |
| right-z-axis-(deg/s)-max    | Recursive Feature Elimination |
| left-z-axis-(deg/s)-max     | Recursive Feature Elimination |
| right_gait_band_energy_mean | Recursive Feature Elimination |
| right_gait_band_energy_std  | Recursive Feature Elimination |
| right-z-axis-(deg/s)-range  | Recursive Feature Elimination |

---

# Machine Learning Model

Given the limited number of data we have to select Machine Learning models in order to get the best possible results. We are going to test a set of machine learning models like:
1. Logistic Regression
2. Random Forest
3. Support Vector Machines

Also we are going to use Leave-One-Out Cross-Validation (LOOCV) for maximum generalization.

In [ ]:
# Reload dataset
df = pd.read_csv('final_dataset.csv')

# Selected top features
selected_features = [
    'right-z-axis-(deg/s)-min',
    'left-z-axis-(deg/s)-iqr',
    'left-z-axis-(deg/s)-min',
    'left-z-axis-(deg/s)-rms',
    'left_gait_band_energy_mean',
    'left-z-axis-(deg/s)-std',
    'right-z-axis-(deg/s)-max',
    'left-z-axis-(deg/s)-max',
    'right_gait_band_energy_mean',
    'right_gait_band_energy_std',
    'right-z-axis-(deg/s)-range'
]

# Feature matrix and labels
X = df[selected_features]
y = df['label']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Initialize LOOCV
cv = LeaveOneOut()

# Define models
models = {
    'Logistic Regression': LogisticRegression(max_iter=100),
    'Random Forest':       RandomForestClassifier(n_estimators=10, random_state=42),
    'SVM (Linear Kernel)': SVC(kernel='linear')
}

# Evaluate models using LOOCV
results = []
for model_name, model in models.items():
    scores = cross_val_score(model, X_scaled, y, cv=cv)
    results.append({'model': model_name, 'mean_accuracy': scores.mean(), 'std_accuracy': scores.std(), 'fold_accuracies': scores.tolist()})

# Display the results
results_df = pd.DataFrame(results)
results_df.to_csv('all_features_model_evaluation_results.csv', index=False)
results_df

## Notes

The result of the training of the models shows that the use of all of these features results at a mean 100% accuracy and 0% standard deviation accuracy. This means that the models over-fit on the data, which would negatively affect the generalizability of the model with unseen or future data. Of course we need to take into consideration that we need to make a model that can be interpreted by medical professionals as well. Based on these criteria we have to choose a subset of features that is helpful to both the patients and the medical staff, with those being: 
1. Right and Left shank maximum swing speed deg/s
2. Right and Left shank minimum swing speed deg/s

In [ ]:
# Reload dataset
df = pd.read_csv('final_dataset.csv')

# Selected top features
selected_features = [
    'right-z-axis-(deg/s)-min',
    'left-z-axis-(deg/s)-min',
    'right-z-axis-(deg/s)-max',
    'left-z-axis-(deg/s)-max'
]

# Feature matrix and labels
X = df[selected_features]
y = df['label']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Initialize LOOCV
cv = LeaveOneOut()

# Define models
models = {
    'Logistic Regression': LogisticRegression(max_iter=100),
    'Random Forest':       RandomForestClassifier(n_estimators=10, random_state=42),
    "SVM (Linear Kernel)": SVC(kernel="linear", probability=True)
}

# Evaluate models using LOOCV
results = []
for model_name, model in models.items():
    scores = cross_val_score(model, X_scaled, y, cv=cv)
    results.append({'model': model_name, 'mean_accuracy': scores.mean(), 'std_accuracy': scores.std(), 'fold_accuracies': scores.tolist()})

# Display the results
results_df = pd.DataFrame(results)
results_df.to_csv('model_evaluation_results.csv', index=False)
results_df

In [ ]:
# Select interpretable features for 2D plot
X = df[['right-z-axis-(deg/s)-min', 'left-z-axis-(deg/s)-min']]
y = df['label']

# Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Create meshgrid for decision boundary
x_min, x_max = X_scaled[:, 0].min() - 1, X_scaled[:, 0].max() + 1
y_min, y_max = X_scaled[:, 1].min() - 1, X_scaled[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300), np.linspace(y_min, y_max, 300))
grid = np.c_[xx.ravel(), yy.ravel()]


# Plot each model's decision boundary
for model_name, model in models.items():
    model.fit(X_scaled, y)
    probs = model.predict_proba(grid)[:, 1].reshape(xx.shape)
    plt.figure(figsize=(10, 7))
    plt.contourf(xx, yy, probs, 25, cmap="coolwarm", alpha=0.8)
    plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c=y, cmap="coolwarm", edgecolors='k', s=100)
    plt.xlabel("right-z-axis-(deg/s)-min (scaled)")
    plt.ylabel("left-z-axis-(deg/s)-min (scaled)")
    plt.title(f"Decision Boundary ({model_name})")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
# Select interpretable features for 2D plot
X = df[['right-z-axis-(deg/s)-max', 'left-z-axis-(deg/s)-max']]
y = df['label']

# Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Create meshgrid for decision boundary
x_min, x_max = X_scaled[:, 0].min() - 1, X_scaled[:, 0].max() + 1
y_min, y_max = X_scaled[:, 1].min() - 1, X_scaled[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300), np.linspace(y_min, y_max, 300))
grid = np.c_[xx.ravel(), yy.ravel()]


# Plot each model's decision boundary
for model_name, model in models.items():
    model.fit(X_scaled, y)
    probs = model.predict_proba(grid)[:, 1].reshape(xx.shape)
    plt.figure(figsize=(10, 7))
    plt.contourf(xx, yy, probs, 25, cmap="coolwarm", alpha=0.8)
    plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c=y, cmap="coolwarm", edgecolors='k', s=100)
    plt.xlabel("right-z-axis-(deg/s)-max (scaled)")
    plt.ylabel("left-z-axis-(deg/s)-max (scaled)")
    plt.title(f"Decision Boundary ({model_name})")
    plt.grid(True)
    plt.tight_layout()
    plt.show()